In [ ]:
from library_dicom.dicom_processor.tools.flip import *
from library_dicom.dicom_processor.tools.create_mip import *
from radiomics.featureextractor import RadiomicsFeatureExtractor 
import os 
import SimpleITK as sitk 
from library_dicom.dicom_processor.tools.threshold_mask import * 

In [ ]:
#get tmtv 

In [ ]:
csv_flip_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/FLIP_metrics_v2.csv'

with open(csv_flip_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [ ]:
old = 'D:FLIP_NIFTI'
new = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

for serie in csv_data : 
    ct_path = '/'.join(serie[5].split('/')[-3:])
    pet_path = '/'.join(serie[6].split('/')[-3:])
    mask_path = '/'.join(serie[7].split('/')[-3:])

    new_ct_path = os.path.join(new, ct_path)
    new_pet_path = os.path.join(new, pet_path)
    new_mask_path = os.path.join(new, mask_path)

    serie[5] = new_ct_path
    serie[6] = new_pet_path 
    serie[7] = new_mask_path

In [ ]:
#Calcul TMTV

In [ ]:
volume = []
for serie in csv_data : 
    print(csv_data.index(serie))
    img = sitk.ReadImage(serie[7])
    mask_array = sitk.GetArrayFromImage(img).transpose()

    pet_img = sitk.ReadImage(serie[6])

    spacing = pet_img.GetSpacing() 

    volume_voxel = spacing[0]*spacing[1]*spacing[2]* 10**(-3)

    pet_array = sitk.GetArrayFromImage(pet_img).transpose()
    mask_array = get_threshold_matrix_4D(mask_array , pet_array, 0.41)

    number_pixel = len(np.where(mask_array != 0)[0])

    volume.append(number_pixel * volume_voxel)


In [ ]:
#Calcul diameter 2D and 3D

In [ ]:
diam_2d = []
diam_3d = []
for row in csv_data : 
    print(csv_data.index(row))
    diameter_2d, diameter_3d = get_diameter(row[7], row[6], int(row[8]))
    diam_2d.append(diameter_2d)
    diam_3d.append(diameter_3d)


In [ ]:
csv_flip_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/FLIP_metrics_v3.csv'

with open(csv_flip_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [ ]:
#correlation plus grand diametre 3d  / tmtv
diametre = []
suv = []
for serie in csv_data : 
    index = csv_data.index(serie)
    diametre.append(float(serie[-2]))
    suv.append(float(serie[-4]))

In [ ]:
import matplotlib.pyplot as plt 

plt.scatter(diametre, suv, alpha=0.7)
plt.xlabel("diamètre 3d")
plt.ylabel("suv")


In [ ]:
from scipy.stats import spearmanr

spearmanr(diametre, suv)

In [ ]:
from scipy.stats import pearsonr 

print(pearsonr(diametre, suv))


In [ ]:
#correlation plus grand diametre 2d  / tmtv
diametre_2d = []
for serie in csv_data : 
    index = csv_data.index(serie)
    diametre_2d.append(float(serie[-3]))

plt.scatter(diametre_2d, suv, alpha=0.7)
plt.xlabel("diamètre 2d")
plt.ylabel("suv")


In [ ]:
spearmanr(diametre_2d, suv)

In [ ]:
pearsonr(diametre_2d, suv)